In [3]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
from neqsim.process import openprocess
import urllib

# Importing the TEG process from file
!wget https://github.com/equinor/neqsimprocess/raw/master/lib/TEGprocess.neqsim

In [2]:
# importing the process from the stored file
# See: https://github.com/equinor/neqsimprocess/blob/master/src/neqsimprocess/glycol/TEGprocess.java
TEGprocess = openprocess('TEGprocess.neqsim')

In [4]:
# Set input parameters
TEGprocess.getUnit("dry feed gas").setTemperature(30.2, 'C')
TEGprocess.getUnit("dry feed gas").setPressure(52.21, "bara")
TEGprocess.getUnit("lean TEG to absorber").setFlowRate(6100.0, "kg/hr")
TEGprocess.getUnit("lean TEG cooler").setOutTemperature(273.15 + 44.85)
TEGprocess.getUnit("rich TEG preheater").setOutTemperature(273.15 + 38.5)
TEGprocess.getUnit("Rich TEG HP flash valve").setOutletPressure(4.9)
TEGprocess.getUnit("rich TEG heater HP").setOutTemperature(273.15 + 62.0)
TEGprocess.getUnit("LP rich glycol heater").setOutTemperature(273.15 + 139.0)
TEGprocess.getUnit("LP rich glycol heater").setOutPressure(1.23)
TEGprocess.getUnit("TEG regeneration column").getReboiler().setOutTemperature(273.15 + 206.6)
TEGprocess.getUnit("stripGas").setPressure(1.23, "bara")
TEGprocess.getUnit("stripGas").setFlowRate(61.88, "kg/hr")
TEGprocess.getUnit("stripGas").setTemperature(80.0, 'C')
TEGprocess.getUnit("lean TEG resycle").getOutletStream().setFlowRate(5500.0, "kg/hr")

#Run the process calculations


JavaObject id=o18

In [ ]:
#Run the process calcs 
TEGprocess.runAsThread() 

In [5]:
#Read results
wetGasRatekghr = TEGprocess.getUnit("water saturated feed gas").getFlowRate("kg/hr")
waterInWetGasppm = TEGprocess.getUnit("water saturated feed gas").getFluid().getPhase(0).getComponent('water').getz()*1.0e6
waterInDryGasppm = TEGprocess.getUnit("dry gas from absorber").getFluid().getPhase(0).getComponent('water').getz()*1.0e6
waterInWetGaskgMSm3 = waterInWetGasppm*0.01802*101325.0/(8.314*288.15)
waterInDryGaskgMSm3 = waterInDryGasppm*0.01802*101325.0/(8.314*288.15)
wtLeanTEGFromReboiler = TEGprocess.getUnit("TEG regeneration column").getLiquidOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
wtLeanTEGFromStripper = TEGprocess.getUnit("TEG stripper").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
reboilerdutykW = TEGprocess.getUnit("TEG regeneration column").getReboiler().getDuty()/1.0e3
condenserdutykW = TEGprocess.getUnit("TEG regeneration column").getCondenser().getDuty()/1.0e3
glycolheaterdutykW = TEGprocess.getUnit("rich TEG preheater").getDuty()/1.0e3
condensertemperature= TEGprocess.getUnit("TEG regeneration column").getCondenser().getTemperature()-273.15
reboilertemperature= TEGprocess.getUnit("TEG regeneration column").getReboiler().getTemperature()-273.15
TEGmakeupkghr = TEGprocess.getUnit("gas to flare").getFluid().getPhase(0).getComponent("TEG").getFlowRate("kg/sec")*3600.0 + TEGprocess.getUnit("dry gas from absorber").getFluid().getPhase(0).getComponent("TEG").getFlowRate("kg/sec")*3600.0 + + TEGprocess.getUnit("gas from degasing separator").getFluid().getPhase(0).getComponent("TEG").getFlowRate("kg/sec")*3600.0
leanTEGflow = TEGprocess.getUnit("TEG absorber").getSolventInStream().getFlowRate("kg/hr")
richTEGflow = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFlowRate("kg/hr")
pump1DutykW = TEGprocess.getUnit("hot lean TEG pump").getEnergy()/1.0e3
pump2DutykW = TEGprocess.getUnit("lean TEG HP pump").getEnergy()/1.0e3
flashGasRate = TEGprocess.getUnit("gas from degasing separator").getFlowRate("kg/hr")
gasToFLareRatekghr = TEGprocess.getUnit("gas to flare").getFlowRate("kg/hr")

In [17]:
# Print results
print('Wet gas rate ', wetGasRatekghr, ' kg/hr')
print('Water in wet gas ', waterInWetGasppm, ' ppm (mol)')
print('Water in dry gas ', waterInDryGasppm, ' ppm (mol)')
print('Water in wet gas ', waterInWetGasppm, ' kg/MSm3')
print('Water in dry gas ', waterInDryGasppm, ' kg/MSm3')
print('wt% TEG from reboiler ', wtLeanTEGFromReboiler, ' wt%')
print('wt% TEG from stripper ', wtLeanTEGFromStripper, ' wt%')
print('Reboiler duty ', reboilerdutykW, ' kW')
print('Condenser duty ', condenserdutykW, ' kW')
print('rich TEG preheater duty ', glycolheaterdutykW, ' kW')
print('temperature from condeser ', condensertemperature, ' C')
print('temperature from reboiler ', reboilertemperature, ' C')
print('TEG makeup ', TEGmakeupkghr, ' kg/hr')
print('lean TEG flow ', leanTEGflow, ' kg/hr')
print('rich TEG flow ', richTEGflow, ' kg/hr')
print('hot lean TEG pump duty ', pump1DutykW, ' kW')
print('lean TEG HP pump duty ', pump2DutykW, ' kW')
print('gas from degasing separator ', flashGasRate, ' kg/hr')
print('gas to flare ', gasToFLareRatekghr, ' kg/hr')

Wet gas rate  390005.21878668887  kg/hr
Water in wet gas  985.3925168003435  ppm (mol)
Water in dry gas  20.18409207246586  ppm (mol)
Water in wet gas  985.3925168003435  kg/MSm3
Water in dry gas  20.18409207246586  kg/MSm3
wt% TEG from reboiler  99.08693991287107  wt%
wt% TEG from stripper  99.86589889281198  wt%
Reboiler duty  486.38261510087045  kW
Condenser duty  -15.838138855344711  kW
rich TEG preheater duty  24.454996889898435  kW
temperature from condeser  104.47344838203458  C
temperature from reboiler  206.60000000000002  C
TEG makeup  0.20739230424968966  kg/hr
lean TEG flow  5499.999999999998  kg/hr
rich TEG flow  5925.4904620936195  kg/hr
hot lean TEG pump duty  2.475936301446869  kW
lean TEG HP pump duty  5.068892282067099  kW
gas from degasing separator  41.36640981023671  kg/hr
gas to flare  147.60234913643382  kg/hr
